# 라이브러리, 한글폰트

In [ ]:
# 데이터 처리 및 분석
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 통계 분석
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, chi2_contingency, f_oneway
from scipy.stats import mannwhitneyu, fisher_exact, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
# import pingouin as pg
# import scikit_posthocs as sp

# 출력 설정
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# 한글 폰트 설정
import platform
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

# 시드 설정
np.random.seed(42)

print("="*60)
print("라이브러리 로드 완료!")
print("한글 폰트 설정 완료!")
print("="*60)


라이브러리 로드 완료!
한글 폰트 설정 완료!


# 매치 데이터 파일 불러온 후 통합

In [2]:
# 티어별 파일과 이름 매핑
tier_files = {
    "Challenger": "TFT_Challenger_MatchData.csv",
    "GrandMaster": "TFT_GrandMaster_MatchData.csv",
    "Master": "TFT_Master_MatchData.csv",
    "Diamond": "TFT_Diamond_MatchData.csv",
    "Platinum": "TFT_Platinum_MatchData.csv"
}

# 데이터프레임 병합
df_list = []
for tier, file in tier_files.items():
    df = pd.read_csv(file, encoding='utf-8')
    df["tier"] = tier
    print(f"[{tier}] 처리 전 데이터 크기: {df.shape}")
    df_list.append(df)

# 통합
df = pd.concat(df_list, ignore_index=True) 
# ignore_index = 기존 인덱스 버리고 새 데이터에 0부터 순차적으로 인덱스 부여
print("="*60)
print("통합 후 데이터 크기:", df.shape)

[Challenger] 처리 전 데이터 크기: (79999, 9)
[GrandMaster] 처리 전 데이터 크기: (80000, 9)
[Master] 처리 전 데이터 크기: (79999, 9)
[Diamond] 처리 전 데이터 크기: (80000, 9)
[Platinum] 처리 전 데이터 크기: (80000, 9)
통합 후 데이터 크기: (399998, 9)


# 챔피언, 아이템 파일 불러오기

In [ ]:
df_champ = pd.read_csv("TFT_Champion_CurrentVersion.csv")
df_item= pd.read_csv("TFT_Item_CurrentVersion.csv")

print(f"[champion]처리 전 데이터 크기: {df_champ.shape}")
print(f"[item]처리 전 데이터 크기: {df_item.shape}")

# 전처리 단계

##### level -> (1~10) 나머지 제거
##### ranked -> (1~8) 나머지 제거
##### Top4 -> 새로운 컬럼 생성 1,0 분류
##### gameDuration,ingameDuration -> 시간 단위 초 -> 분
##### champion,combination -> {},공백,NaN 제거

In [3]:
# 원본 보존
df_raw = df.copy()

# 1) level 필터링
df_level = df_raw[(df_raw['level'] >= 1) & (df_raw['level'] <= 10)].copy()
print("✅ level 기준 필터링 완료")
print(f"남은 데이터 개수: {len(df_level):,}행")
print(f"제거된 이상치 개수: {len(df_raw) - len(df_level):,}행\n")

# 2) Ranked 필터링 (level 필터링 이후 데이터 기준)
df = df_level[(df_level['Ranked'] >= 1) & (df_level['Ranked'] <= 8)].copy()
print("✅ Ranked 기준 필터링 완료")
print(f"남은 데이터 개수: {len(df):,}행")
print(f"제거된 이상치 개수: {len(df_level) - len(df):,}행\n")

# 3) 순위 그룹화 (⭐ 필터 이후에 다시 생성)
df["Top4"] = (df["Ranked"] <= 4)

top4_counts = df['Top4'].value_counts()
print("✅ Top4 생성 완료")
print(top4_counts)
print(f"Top4 비율: {top4_counts.get(True, 0) / len(df) * 100:.2f}%\n")

# 4) 시간 초단위 -> 분단위 변경
df['gameDuration'] = (df['gameDuration'] // 60).astype(int)
df['ingameDuration'] = (df['ingameDuration'] // 60).astype(int)
print("✅ gameDuration / ingameDuration 분단위 변환 완료\n")


# 5) champion, combination: "{}" 문자열, 빈문자열, NaN 제거
before = len(df)  # 필터 전 행 개수 저장

df = df.dropna(subset=['champion', 'combination']).copy()
df = df[
    (df['champion'].str.strip() != '{}') &
    (df['champion'].str.strip() != '') &
    (df['combination'].str.strip() != '{}') &
    (df['combination'].str.strip() != '')
].copy()

after = len(df)  # 필터 후 행 개수
removed = before - after  # 제거된 행 수 계산

print("✅ '{}' 문자열/빈문자열 제거 완료")
print(f"제거된 행 개수: {removed:,}행")
print(f"남은 데이터 개수: {after:,}행\n")


# 6) 전체 제거 행 수 계산
removed_total = len(df_raw) - len(df)
print(f"🚨 전체 제거된 행 개수: {removed_total:,}행")
print(f"✅ 최종 남은 행 개수: {len(df):,}행")

✅ level 기준 필터링 완료
남은 데이터 개수: 399,998행
제거된 이상치 개수: 0행

✅ Ranked 기준 필터링 완료
남은 데이터 개수: 399,906행
제거된 이상치 개수: 92행

✅ Top4 생성 완료
Top4
False    199959
True     199947
Name: count, dtype: int64
Top4 비율: 50.00%

✅ gameDuration / ingameDuration 분단위 변환 완료

✅ '{}' 문자열/빈문자열 제거 완료
제거된 행 개수: 422행
남은 데이터 개수: 399,484행

🚨 전체 제거된 행 개수: 514행
✅ 최종 남은 행 개수: 399,484행
